In [1]:
import sys
import os
import argparse

import carla

from environment.env import CarlaEnv
from common.loggers.comet_logger import CometLogger
from projects.morel_mopo.config.logger_config import CometLoggerConfig
from projects.morel_mopo.algorithm.mopo import MOPO
from projects.morel_mopo.config.morel_mopo_config import DefaultMLPMOPOConfig, DefaultProbMLPMOPOConfig, DefaultProbGRUMOPOConfig, DefaultMLPObstaclesMOPOConfig

In [2]:
exp_name = 'test'
gpu = 3

In [3]:
logger_conf = CometLoggerConfig()
logger_conf.populate(experiment_name = exp_name, tags = ["MOPO"])
logger_conf.experiment_key = 'cb248318a263406795aaabc4e8a51a3f'
logger = CometLogger(logger_conf)

config = DefaultMLPObstaclesMOPOConfig()
config.populate_config(gpu = gpu, policy_algorithm = "PPO")

model = MOPO.load(logger, 'best_model_700000.zip', gpu, policy_only=True, config=config)

Verified config CometLoggerConfig. Note: this just checks for missing values!
LOGGER: Could not find experiment in log_dir. Creating new log directory.
LOGGER: Experiment name - uncertain_1
LOGGER: Created log directory at /home/scratch/brianyan/comet_logs/comet_temp/uncertain_1
reward config Simple2RewardConfig
Verified config DefaultDynamicsModuleConfig. Note: this just checks for missing values!
Verified config DefaultMLPDynamicsEnsembleConfig. Note: this just checks for missing values!
Verified config ObstaclesMixedDeterministicMLPDataModuleConfig. Note: this just checks for missing values!
Verified config ObstaclesMLPDynamicsConfig. Note: this just checks for missing values!
Verified config DefaultDynamicsModuleConfig. Note: this just checks for missing values!
Verified config DefaultMLPDynamicsEnsembleConfig. Note: this just checks for missing values!
Verified config ObstaclesMixedDeterministicMLPDataModuleConfig. Note: this just checks for missing values!
Verified config Obstacl

/zfsauton2/home/brianyan/anaconda3/envs/carla/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
env = CarlaEnv(config=model.eval_env_config, logger=logger, log_dir=logger.log_dir)

  0%|          | 0/30 [00:00<?, ?it/s]

Attempting to start carla on GPU 3
Launched server at port: 19129
Waiting 30s for server to finish setting up


100%|██████████| 30/30 [00:30<00:00,  1.00s/it]


server_version 0.9.10.1-dirty


In [5]:
successes = 0
for i in range(25):
    obs = env.reset(index=i, unseen=True)
    total_reward = 0

    for _ in range(10000):
        action = model.policy_predict(obs)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        if done:
            break

    if info['termination_state'] == 'success':
        successes += 1

    print(total_reward)
    print(info['termination_state'])

print(successes)

ValueError: Error: Unexpected observation shape (1, 6) for Box environment, please use (1, 4) or (n_env, 1, 4) for the observation shape.

In [6]:
env.close()

Killed server process
